In [1]:
%load_ext autoreload
%autoreload 2

import tqdm, sys, os, time
import pandas as pd
import scipy as sp

import scanpy as sc
import anndata as ad

# Load scRNAseq data

In [2]:
adata = ad.read_h5ad("cache/data-raw-snRNAseq_10x_v3_Broad-matrix.h5ad").transpose()
adata

AnnData object with n_obs × n_vars = 215823 × 31053

## Load samples

In [22]:
obs = pd.read_csv("data/raw/snRNAseq_10x_v3_Broad/barcodes.csv.gz", 
                  compression='gzip', skiprows=1,
                  names=['sample_name'], index_col=0)


metadata = pd.read_csv("data/raw/snRNAseq_10x_v3_Broad/sample_metadata.csv", 
                       dtype={'dataset': 'category',
                              'QC': 'category',
                              'Gender': 'category',
                              'Allen.cluster_id': 'str',
                              'comb.QC': 'category',
                             },
                       # parse_dates=['Lib_Date', 'Amp_Date'],
                       index_col='sample_name')
metadata.drop(columns=['Unnamed: 0'], inplace=True)


cluster_membership = pd.read_csv("data/raw/snRNAseq_10x_v3_Broad/cluster.membership.csv", 
                                 names=['barcode', 'cluster_id'], 
                                 dtype={'cluster_id': 'category'},
                                 skiprows=1, index_col=0)

cluster_annotation = pd.read_csv("data/raw/snRNAseq_10x_v3_Broad/cluster.annotation.csv", 
                                 index_col='cluster_id', dtype={'cluster_id': 'category'})

cluster_membership = cluster_membership.join(cluster_annotation, on='cluster_id')


obs = obs.join(metadata).join(cluster_membership)
obs.describe(percentiles=[])

,nUMI,nGene,size,gene.counts,umi.counts,Broad.QC.doublet,Broad.QC.Mito,Broad.passQC,MALE,cl
count,215823.000000,215823.000000,174348.000000,174348.000000,174348.000000,174348.000000,174348.000000,174348.000000,174348.000000,174348.000000
mean,11933.397219,3693.444721,10881.678344,3995.986326,12818.433587,0.043944,0.027260,0.927649,0.445328,93.735609
std,9612.197936,1947.415193,8103.741188,1474.187164,7155.579677,0.111508,0.158758,0.189136,0.078705,36.085813
min,501.000000,210.000000,14.000000,1409.400000,2308.564103,0.000000,0.000000,0.000000,0.000000,1.000000
50%,10228.000000,3882.000000,14450.000000,4481.203891,13449.868170,0.010000,0.000000,0.990000,0.448615,94.000000
max,104062.000000,10829.000000,23971.000000,7026.779330,34730.418990,1.000000,1.000000,1.000000,1.000000,161.000000


In [35]:
adata.obs = obs

## Load variables

In [33]:
var = pd.read_table("data/raw/snRNAseq_10x_v3_Broad/features.csv").set_index("gene")
var.index.name = 'gene_name'
adata.var = var
var

""
gene_name
Xkr4
Gm1992
Gm37381
Rp1
Sox17
...
AC168977.1
AC149090.1
CAAA01118383.1


## AnnData 

In [36]:
adata

AnnData object with n_obs × n_vars = 215823 × 31053
    obs: 'nUMI', 'nGene', 'dataset', 'QC', 'cluster', 'Allen.cluster_id', 'Allen.cluster_label', 'Allen.cluster_color', 'Allen.class_label', 'Allen.subclass_label', 'comb.QC', 'cluster_id', 'cluster_label', 'cluster_color', 'class_label', 'subclass_label', 'size', 'gene.counts', 'umi.counts', 'Broad.QC.doublet', 'Broad.QC.Mito', 'Broad.passQC', 'MALE', 'Comb.QC', 'cl'

In [37]:
adata.obs[['cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size']]

,cluster_id,cluster_label,subclass_label,class_label,cluster_color,size
sample_name,,,,,,
pBICCNsMMrMOpRMiF007d190314_AAAGAACGTTAATGAG,NaN,NaN,NaN,NaN,NaN,NaN
pBICCNsMMrMOpRMiF007d190314_AAAGAACTCTACGGTA,48,L2/3 IT_1,L2/3 IT,Glutamatergic,#6F836B,23971.0
pBICCNsMMrMOpRMiF007d190314_AAAGGGCAGAGGCTGT,NaN,NaN,NaN,NaN,NaN,NaN
pBICCNsMMrMOpRMiF007d190314_AAAGGGCAGGGCTAAC,91,Astro Aqp4_1,Astro,Non-Neuronal,#7C2D1F,16143.0
pBICCNsMMrMOpRMiF007d190314_AAAGGTAGTGGCTGAA,48,L2/3 IT_1,L2/3 IT,Glutamatergic,#6F836B,23971.0
...,...,...,...,...,...,...
pBICCNsMMrMOpRMiM006d190320_TTTGGTTGTATCGCTA,NaN,NaN,NaN,NaN,NaN,NaN
pBICCNsMMrMOpRMiM006d190320_TTTGGTTTCGCAAGAG,90,Oligo Opalin_4,Oligo,Non-Neuronal,#474662,16566.0
pBICCNsMMrMOpRMiM006d190320_TTTGTTGAGACTCTTG,51,L5 IT S100b,L5 IT,Glutamatergic,#00CF1E,8684.0


# Preprocessing

In [38]:
sc.tl.pca(adata, svd_solver='arpack')

In [40]:
adata

AnnData object with n_obs × n_vars = 215823 × 31053
    obs: 'nUMI', 'nGene', 'dataset', 'QC', 'cluster', 'Allen.cluster_id', 'Allen.cluster_label', 'Allen.cluster_color', 'Allen.class_label', 'Allen.subclass_label', 'comb.QC', 'cluster_id', 'cluster_label', 'cluster_color', 'class_label', 'subclass_label', 'size', 'gene.counts', 'umi.counts', 'Broad.QC.doublet', 'Broad.QC.Mito', 'Broad.passQC', 'MALE', 'Comb.QC', 'cl'
    uns: 'pca'
    obsm: 'X_pca'
    varm: 'PCs'

# Save

In [41]:
adata.write_loom("data/processed/snRNAseq_10x_v3_Broad.loom", write_obsm_varm=True)

/work/03637/nct7456/ls6/micromamba/lib/python3.9/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/work/03637/nct7456/ls6/micromamba/lib/python3.9/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/work/03637/nc